## 01. 기본정보 셋팅

In [9]:

from lib.custom_confluence import ConfluenceLoader
# from langchain_community.document_loaders import ConfluenceLoader
# from langchain.document_loaders.confluence import ContentFormat
from langchain.text_splitter import CharacterTextSplitter
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

url = os.getenv("CONFLUENCE_URL")
username = os.getenv("CONFLUENCE_USERNAME")
api_key = os.getenv("CONFLUENCE_API_KEY")




## 02.Custom Conflence Loader로 OCR + Bedrock 처리 후 정보 가져오기

In [2]:

# Confluence 접속 정보
loader = ConfluenceLoader(
    url=url,
    username=username,
    api_key=api_key,
    # space_key="SD",
    page_ids=['63897603'],  # 로드할 페이지의 ID
    include_attachments=True,
    ocr_languages="eng+kor",
    keep_markdown_format=True
    # content_format=ContentFormat.EDITOR
    # limit=500,
)
documents = loader.load()
# print(documents)


CD_CNTNT CMM_CD CD_VAL 
1 Sales 22 일반 
2 Sales 23 오늘도착 
3 Sales 90 S-VIP 
4 Sales 91 배송불필요:수거지시 후 반품취소 
5 Sales 93 맞교환 주문 
6 Sales 97 배송불필요 
7 Sales 9H 도착일 선택 
8 Sales 9N 식품당일배송 
9 Sales 9S 재고범위내 출하 
10 Sales 9T 선출 
11 Sales 9Y 요우커지정일 
12 Sales 9Z 요우커일반 
13 Return 01 수거필요 
14 Return 02 수거불필요:택배사분실 
15 Return 03 수거불필요:택배사분실 
16 Return 04 수거불필요:업체 입고완료건 
17 Return 05 수거불필요:카드도용고객 
18 Return 06 수거불필요:AS불가 
19 Return 07 수거불필요:가주문반품건 
20 Return 08 수거불필요:기타 
21 Return 09 맞교환 반품 
22 Return OA 수거불필요:고객직접발송 
23 Return 10 반품취소 
24 Return 11 수취거절:배송전 취소 
25 Return 12 수거지정일 
26 Return 13 편의점반품 
27 Return 14 수거불필요:편의점분실 
28 Return 92 수거볼필요:수거지시 후 반품취소


## 03. Markdown TextSplitter로 데이터 분할하기

In [3]:

from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter

# 마크다운 헤더를 기준으로 분할
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]
)

# 문자 수를 기준으로 분할
char_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# 1. 각 Page 별로 split 수행후 
# 2. Markdown header기준으로 분리하고
# 3. 최종 배열에 Page Content / Markdown Metadata / Conflent Page Metadata 입력
final_splits = []
for document in documents:
    # print(document.page_content)
    # print(doc.metadata)
    
    # markdown 데이터 분할
    markdown_splits = markdown_splitter.split_text(document.page_content)
    # print(markdown_splits)
    
    for markdown_split in markdown_splits:
        final_splits.append({
                    "page_content": markdown_split.page_content,
                    "metadata":{
                        "markdown_metadata": markdown_split.metadata,
                        "document_metadata": document.metadata
                    }
        })
        
for i, split in enumerate(final_splits):
    print(f"Split {i + 1}:\n{split}\n")



Split 1:
{'page_content': 'note이 템플릿은 온라인 회의 관리 도구 Meetical에서 제공합니다.  \n이 템플릿은 온라인 회의 관리 도구 Meetical에서 제공합니다.  \n| **날짜** | //를 입력하여 날짜를 추가 |\n| --- | --- |\n| **시간** | 예: 오전 10:00~10:30 KST |\n| **참여자** | 참가자 @ 멘션 |\n| **위치** | /link를 입력하여 화상 회의 링크를 추가 |\n| **회의 유형** | 예: 주간 체크인 회의 |\n| **자료** | /링크를 입력하여 이전 미팅 메모 또는 기타 자료를 추가 |  \n재미 있고 편안한 분위기에서 회의를 시작하여 대화를 준비합니다. 그런 다음 목표를 향한 진행 상황에 대해 논의하기 전에 서로 열린 질문을 하는 시간을 가집니다.', 'metadata': {'markdown_metadata': {}, 'document_metadata': {'title': '페이지내 이미지 테스트', 'id': '63897603', 'source': 'https://lge-web-project.atlassian.net/wiki/spaces/SD/pages/63897603', 'when': '2024-06-02T01:32:45.382Z'}}}

Split 2:
{'page_content': '회의 전에 표를 채웁니다. 지난 회의 이후 배운 점과 다음 회의에서 달성하려는 점에 대한 질문을 추가합니다.  \n| **참가자** | **마지막으로 만난 이후의 우선 순위** | **다시 만날 때까지의 우선 순위** |\n| --- | --- | --- |\n| 첫 번째 참가자 @멘션 | * 예: 마지막 만난 이후에 프로젝트에 대해 무엇을 알게 되었습니까? * | * 예: 다음 주 영업 프레젠테이션을 어떻게 준비해야 합니까? * |\n| 두 번째 참가자 @ 멘션 | * 예: 최근에 작업의 속도를 늦추거나 작업에 방해가 된 요인이 있습니까? * | * 예: 분기별 목표에 

## 04. AOSS indexing 진행하기 

In [6]:
from lib.bedrock import get_embedding_output

data_list = []
embedding_dimension = 1024

for doc in final_splits:
    content = doc["page_content"]
    meta = doc["metadata"]
    embedding = get_embedding_output(content)
    
    if embedding and len(embedding) == embedding_dimension:
        data_list.append({
            "content": content,
            "content_embeddings": embedding,
            "metadata": meta,
        })
        print("Success to get index")
    else:
        print(f"Error: {content}")

Success to get index
Success to get index
Success to get index
Success to get index


In [11]:
from lib.opensearch import getOpenSearchClient
aoss_client = getOpenSearchClient()

vector_index_name = os.getenv("AOSS_VECTOR_INDEX")

In [12]:
for data in data_list:
    try:
        response = aoss_client.index(index=vector_index_name, body=data)
        print(response)
    except Exception as e:
        print(f"Error: {e}")

{'_index': 'rag-hol-index-vector', '_id': '1%3A0%3ADHdtaJABoZyGEX8r3Mx0', '_version': 1, 'result': 'created', '_shards': {'total': 0, 'successful': 0, 'failed': 0}, '_seq_no': 0, '_primary_term': 0}
{'_index': 'rag-hol-index-vector', '_id': '1%3A0%3AnJltaJABwAR7yjnm3_i3', '_version': 1, 'result': 'created', '_shards': {'total': 0, 'successful': 0, 'failed': 0}, '_seq_no': 0, '_primary_term': 0}
{'_index': 'rag-hol-index-vector', '_id': '1%3A0%3ADXdtaJABoZyGEX8r4czY', '_version': 1, 'result': 'created', '_shards': {'total': 0, 'successful': 0, 'failed': 0}, '_seq_no': 0, '_primary_term': 0}
{'_index': 'rag-hol-index-vector', '_id': '1%3A0%3AnZltaJABwAR7yjnm4_ii', '_version': 1, 'result': 'created', '_shards': {'total': 0, 'successful': 0, 'failed': 0}, '_seq_no': 0, '_primary_term': 0}
